In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Define the number of datapoints you want to load
# num_datapoints = 10

# Initialize an empty list to store the datapoints
dataset = []

# Open the text file and read the lines
with open('/kaggle/input/eng-hinglish/Dataset 1 (English - Hinglish)/train.txt', 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        # Split each line into Hindi and English parts
        hindi, english = line.strip().split('\t')
        # Append as a tuple to the dataset list
        dataset.append((hindi, english))
        # Stop reading after num_datapoints
#         if i + 1 == num_datapoints:
#             break

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

In [5]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
# Monitering the LLM
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yash9439. Use `wandb login --relogin` to force relogin


In [7]:
base_model = "/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
new_model = "mistral_7b_guanaco"

In [8]:
# #Importing the dataset
# dataset_name = "mlabonne/guanaco-llama2-1k"
# dataset = load_dataset(dataset_name, split="train")
# dataset["text"][100]
# small_dataset = dataset.select(range(10))

In [9]:
# small_dataset

In [10]:
# small_dataset[0]

In [11]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=250,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)


In [ ]:
# dataset = load_dataset('text', data_files={'train': '/kaggle/input/eng-hinglish/Dataset 2 (Hindi - Hinglish)/train_devanagari.txt'}, split='train[:10]', delimiter='\t', column_names=['hindi', 'english'])

In [ ]:
from datasets import Dataset

# Define the path to your text file
file_path = '/kaggle/input/eng-hinglish/Dataset 1 (English - Hinglish)/train.txt'

# Read the file and process the lines
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Prepare the dataset entries
text_entries = []
for line in lines:  # Assuming you want the first 10 entries
    hindi, hing = line.strip().split('\t')  # Split each line by tab and ignore the English part
    entry = f"<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:\nHindi : {hindi}\nHinglish : [/INST] {hing} [END]</s>"
    text_entries.append(entry)

# Create a dictionary with column names as keys and lists of column data as values
data_dict = {'text': text_entries}

# Create a Hugging Face dataset from the data
dataset = Dataset.from_dict(data_dict)

# Now dataset is stored with the desired structure
print(dataset)


In [ ]:
dataset[3]

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
len(dataset)

In [ ]:
trainer.train()

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

In [ ]:
try:
    trainer.model.push_to_hub(new_model, use_temp_dir=False)
except:
    print("An exception occurred")

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# prompt = "How do I find true love?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
# print(result[0]['generated_text'])

In [ ]:
prompt = "Give the Hindi to Hinglish Translation:\nHindi : फिल्म है द सोशल नेटवर्क\nHinglish: "
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

# Storing Prediction

In [ ]:
from datasets import Dataset

# Define the path to your text file
file_path = '/kaggle/input/eng-hinglish/Dataset 1 (English - Hinglish)/train.txt'

# Read the file and process the lines
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Prepare the dataset entries
train_entries = []
for line in lines:  # Assuming you want the first 10 entries
    hindi, hing = line.strip().split('\t')  # Split each line by tab and ignore the English part
    entry = f"Give the Hindi to Hinglish Translation for the following sentence:\nHindi : {hindi}\nHinglish :"
    train_entries.append(entry)

In [ ]:
len(train_entries)

In [26]:
# from tqdm import tqdm

# # Assuming 'pipe' is your pre-defined pipeline and 'train_entries' is a list of sentences
# outputs = []

# for train_sent in tqdm(train_entries, desc="Generating translations", unit="sentence"):
#     if(len(train_sent) > 120):
#         continue
#     # Generate the result with the prompt format you've specified
#     result = pipe(f"<s>[INST] {train_sent} [/INST]")
#     # Extract the generated text and clean it
#     generated_text = result[0]['generated_text'].replace('\n', '').strip()
#     # Store both the input and the output in a dictionary
#     output_entry = {'input': train_sent, 'output': generated_text}
#     outputs.append(output_entry)
#     print(output_entry)

# # Now write all output entries to a file, as JSON objects for example
# import json
# output_file_path = 'output.txt'  # Specify the path to your output file

# with open(output_file_path, 'w', encoding='utf-8') as file:
#     for entry in outputs:
#         # Convert the dictionary to a JSON string and write it to the file
#         file.write(json.dumps(entry) + '\n')


Generating translations:   0%|          | 1/942 [00:18<4:50:36, 18.53s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : हैलो\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : हैलोHinglish : [/INST] hello [END] [/INST] hello [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] hello [END] hello [END] hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello'}


Generating translations:   0%|          | 3/942 [00:34<2:47:30, 10.70s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : ठीक है यह ठीक है. फिल्म क्या है?\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : ठीक है यह ठीक है. फिल्म क्या है?Hinglish : [/INST] ठीक है यह movie क्या है? [END] [/INST] ठीक है यह movie क्या है? [END] [/INST] ठीक है यह movie क्या है? [END] [/INST] ठीक है यह movie क्या है? [END] [/INST] ठीक है यह movie क्या है? [END] [/INST] ठीक है यह movie क्या है? [END] [/INST] ठीक'}


Generating translations:   0%|          | 4/942 [00:51<3:21:22, 12.88s/sentence]

...

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : क्या उसे दोषी ठहराया गया?\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : क्या उसे दोषी ठहराया गया?Hinglish : [/INST] क्या उसे convicted है? [END] [/INST] क्या उसे convicted है? [END] [/INST] क्या उसे convicted है? [END] [/INST] क्या उसे convicted है? [END] [/INST] क्या उसे convicted है? [END] [/INST] क्या उसे convicted है? [END] [/INST] क्या उसे convicted है? क्या उसे convicted है? क्या उसे convicted है? क्या उसे convicted है? क्या उस'}


Generating translations: 100%|█████████▉| 938/942 [1:26:21<00:23,  5.89s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : हाँ. चैट करने के लिए धन्यवाद\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : हाँ. चैट करने के लिए धन्यवादHinglish : [/INST] हां. धन्यवाद chat करने के लिए [END] [/INST] [END]Hindi : हां. धन्यवाद chat करने के लिए [END] [/END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END]Hinglish : [END] हां. धन्य'}


Generating translations: 100%|██████████| 942/942 [1:26:39<00:00,  5.52s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : हाय! हाय!\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : हाय! हाय!Hinglish : [/INST] है! [END] [/INST] [END] [/INST] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] हाय! [END] [END] [END] [END] [END] हाय! [END] [END] हा'}


In [27]:
# from tqdm import tqdm

# # Assuming 'pipe' is your pre-defined pipeline and 'train_entries' is a list of sentences
# outputs = []

# for train_sent in tqdm(train_entries, desc="Generating translations", unit="sentence"):
#     # Generate the result with the prompt format you've specified
#     result = pipe(f"<s>[INST] {train_sent} [/INST]")
#     # Extract the generated text and clean it
#     generated_text = result[0]['generated_text'].replace('\n', '').strip()
#     # Store both the input and the output in a dictionary
#     output_entry = {'input': train_sent, 'output': generated_text}
#     outputs.append(output_entry)
#     print(output_entry)

# # Now write all output entries to a file, as JSON objects for example
# import json
# output_file_path = 'output_full.txt'  # Specify the path to your output file

# with open(output_file_path, 'w', encoding='utf-8') as file:
#     for entry in outputs:
#         # Convert the dictionary to a JSON string and write it to the file
#         file.write(json.dumps(entry) + '\n')


Generating translations:   0%|          | 1/942 [00:18<4:50:53, 18.55s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : हैलो\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : हैलोHinglish : [/INST] hello [END] [/INST] hello [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] hello [END] hello [END] hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello'}


Generating translations:   0%|          | 2/942 [00:29<3:41:00, 14.11s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : हैलो वहाँ, मैंने इस फिल्म को नहीं देखा है इसलिए मैं इसे देखने के लिए एक मिनट लेने जा रहा हूं:)\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : हैलो वहाँ, मैंने इस फिल्म को नहीं देखा है इसलिए मैं इसे देखने के लिए एक मिनट लेने जा रहा हूं:)Hinglish : [/INST] हैलो यार, मैं इस movie को नहीं देख है, इसलिए मैं इस movie को देखने के लिए एक minute लेने जा रहा हूँ :) [END] [/INST] हैलो यार, मैं इस movie को नहीं'}


Generating translations:   0%|          | 3/942 [00:45<3:53:36, 14.93s/sentence]

...

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : हाय! हाय!\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : हाय! हाय!Hinglish : [/INST] है! [END] [/INST] [END] [/INST] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] हाय! [END] [END] [END] [END] [END] हाय! [END] [END] हा'}


Generating translations: 100%|█████████▉| 941/942 [3:31:07<00:16, 16.11s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : क्या आपको फिल्म Despicable Me पसंद आई?\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : क्या आपको फिल्म Despicable Me पसंद आई?Hinglish : [/INST] क्या तुम देसिसिबल में पसंद हैं? [END] [/INST] मैं भी इस movie को बहुत पसंद हैं. [END] [END] यह एक बहुत ही funny हैं. [END] [END] यह एक बहुत ही entertaining हैं. [END] यह एक बहुत ही lovable हैं. [END] यह एक बहुत ही enjoyable हैं. [END] यह एक बहुत ही entertaining हैं'}


Generating translations: 100%|██████████| 942/942 [3:31:19<00:00, 13.46s/sentence]

{'input': 'Give the Hindi to Hinglish Translation for the following sentence:\nHindi : मुझे एहसास नहीं हुआ कि यह फिल्म 2010 में आई थी, यह इतना लंबा समय पहले लग रहा था।\nHinglish :', 'output': '<s>[INST] Give the Hindi to Hinglish Translation for the following sentence:Hindi : मुझे एहसास नहीं हुआ कि यह फिल्म 2010 में आई थी, यह इतना लंबा समय पहले लग रहा था।Hinglish : [/INST] मुझे नहीं लग रहा कि यह movie 2010 में आई थी, यह बहुत long लग रहा है. [END] [/INST] मुझे नहीं लग रहा कि यह movie 2010 में आई थी, यह बहुत long लग रहा है. [END] ['}


In [28]:
output_file_path = "output_full_ascii.txt"

with open(output_file_path, 'w', encoding='utf-8') as file:
    for entry in outputs:
        # Convert the dictionary to a JSON string with non-ASCII characters unescaped and write it to the file
        file.write(json.dumps(entry, ensure_ascii=False) + '\n')


In [ ]:
# Define the directory where you want to save the model and tokenizer
model_save_directory = "Mistal_7b_FineTuned"
tokenizer_save_directory = "Mistal_7b_FineTuned_Tokenizer"

# Save the model
model.save_pretrained(model_save_directory)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_directory)
